# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [70]:
tf.__version__

'2.3.1'

### Importar o dataset

In [71]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [72]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [73]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [75]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [136]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [137]:
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))

### Adding the second hidden layer

In [138]:
ann.add(tf.keras.layers.Dense(units=16, activation='sigmoid'))

In [139]:
ann.add(tf.keras.layers.Dense(units=8, activation='softmax'))

### Adding the output layer

In [140]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))


# Part 3 - Training the ANN

### Compiling the ANN

In [141]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [142]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4718 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4378 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4616 - accuracy: 0.8109
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4311 - accuracy: 0.8091
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4227 - accuracy: 0.8174
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8250
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4000 - accuracy: 0.8292
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8329
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3829 - accuracy: 0.8354
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3795 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [143]:
y_pred = ann.predict(X_test)
y_pred

array([[0.3204477 ],
       [0.20938201],
       [0.06725831],
       ...,
       [0.16721393],
       [0.1512676 ],
       [0.23573305]], dtype=float32)

In [144]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [145]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[1504   91]
 [ 200  205]]


In [146]:
Tp, Fp, Fn, Tn = cm.reshape(1, -1, order='F').squeeze()
# https://ccrma.stanford.edu/workshops/mir2009/references/ROCintro.pdf
accuracy = (Tp+Tn) / (Tp+Fp+Tn+Fn)
precision = (Tp) / (Tp+Fp)
recall = (Tp) / (Tp+Fn)
f1_score = 2 * (precision*recall) / (precision+recall)
metrics_names = ["Accuracy", "Precision", "Recall", "F1 Score"]
metrics = [accuracy, precision, recall, f1_score]
session_results = pd.DataFrame(metrics, index=metrics_names)
print(session_results)

                  0
Accuracy   0.854500
Precision  0.882629
Recall     0.942947
F1 Score   0.911791


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)